In [2]:
import os
import shutil
import random
import json
from midi2audio import FluidSynth

import numpy as np
import tensorflow as tf
from tensorflow import keras

import pydub
from pydub import AudioSegment

import librosa
import random

In [2]:
available_list = {}
train_list = []
test_list = []

with open(r'C:\Users\Arnold\Desktop\4560\CNN\dataset\400-100-dataset\list.json', 'r') as f:
    available_list = json.load(f)

train_list = available_list['train']
test_list = available_list['test']

In [3]:
work_dir = r'C:\Users\Arnold\Desktop\4560\CNN\dataset\mid-aud-to-midi-bearaudiotool.com-mp3-to-midi'
train_dir = r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\400-100\train'
test_dir =  r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\400-100\test'

for parent, dirnames, filenames in os.walk(work_dir,  followlinks=True):
    for filename in filenames:
        if filename.endswith(".mid"):
            index = os.path.splitext(os.path.splitext(filename)[0])[0]
            if str(index) in train_list:
                shutil.copy(parent+'\\'+filename, train_dir+'\\'+filename)
            if str(index) in test_list:
                shutil.copy(parent+'\\'+filename, test_dir+'\\'+filename)  

In [25]:
build_train_dataset = r'convert_dir_to_note_sequences  --input_dir='+train_dir+r' --output_file='+train_dir+r'\_train.tfrecord  --recursive'
build_test_dataset  = r'convert_dir_to_note_sequences  --input_dir='+test_dir +r' --output_file='+test_dir +r'\_test.tfrecord  --recursive'

print(os.popen(build_test_dataset).readlines())
print(os.popen(build_train_dataset).readlines())

[]
[]


In [6]:
config = r'cat-mel_2bar_small'
run_dir = r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\400-100'+'\\'+config+'\\'
train_dir = r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\400-100\train'
checkpoint_file = r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\400-100\cat-mel_2bar_small\train'

In [7]:
# vae_train = r'music_vae_train \
# --config='+config+r' \
# --run_dir='+run_dir+r' \
# --mode=train \
# --tfds_name='+ train_dir +r'\_train.tfrecord'

# print(os.popen(vae_train).readlines())

# it would not work, run it by .bat and cmd.

In [6]:
# here should be a midi split loop. However cannot find such one.
interMusic_1 = r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\59.mp3.mid'
interMusic_2 = r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\89.mp3.mid'
output_dir = r'C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\output'

In [15]:
music_interpolate = r'music_vae_generate \
--config='+config+r' \
--checkpoint_file='+checkpoint_file+r' \
--mode=interpolate \
--num_outputs=5 \
--input_midi_1='+interMusic_1 +r' \
--input_midi_2='+interMusic_2 +r' \
--output_dir='+output_dir

print(os.popen(music_interpolate).readlines())

['5 valid inputs extracted from `C:\\Users\\Arnold\\Desktop\\4560\\MusicVAE\\DEAM\\practice\\59.mp3.mid`. Outputting these potential inputs as `C:\\Users\\Arnold\\Desktop\\4560\\MusicVAE\\DEAM\\practice\\output\\cat-mel_2bar_small_input1-extractions_2020-10-16_010019-*-of-005.mid`. Call script again with one of these instead.\n']


In [36]:
for parent, dirnames, filenames in os.walk(output_dir,  followlinks=True):
    for filename in filenames:
        if filename.endswith(".mid"):
            index = os.path.splitext(filename)[0]
            print(os.path.join(output_dir, filename))
            FluidSynth().midi_to_audio(os.path.join(output_dir, filename),os.path.join(output_dir,index+'.wav'))

C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\output\cat-mel_2bar_small_input1-extractions_2020-10-16_010019-000-of-005.mid


FileNotFoundError: [WinError 2] 系统找不到指定的文件。

In [4]:
model = keras.models.load_model(r'C:\Users\Arnold\Desktop\4560\CNN\best_crnn')

In [8]:

for parent, dirnames, filenames in os.walk(output_dir,  followlinks=True):
    for filename in filenames:
        if filename.endswith(".wav"):
            index = os.path.splitext(filename)[0]
            print(os.path.join(output_dir, filename))
            wavsong = AudioSegment.from_wav(os.path.join(output_dir, filename))
            
            one_data = []
            test_data = []

            # mel freq
            for i in range(4):
                outputpath = os.path.join(output_dir,index+'-'+str(i)+".wav")
                wavsong[(500*i):(500+500*i)].export(outputpath, format="wav")
                y, sr = librosa.load(outputpath, sr=None)
                os.remove(outputpath)

                fragment = np.array(
                                np.log(
                                    librosa.feature.melspectrogram(
                                        y, sr, n_mels=128,n_fft=int(sr*0.5), hop_length=int(sr*1)
                                    )
                                ).transpose())
                fragment = fragment[np.newaxis,:]

                if one_data==[]:
                    one_data = fragment.copy()                    
                else:
                    one_data = np.vstack([one_data,fragment.copy()])
            
            # self repliction
            for i in range(15):
                if test_data==[]:
                    test_data = one_data.copy()                    
                else:
                    test_data = np.vstack([one_data,test_data.copy()])                    
                    
            # crnn test
            test_data = test_data.reshape(1,60,128,1)
            predict_va = model.predict(test_data)
            print(sum(sum(predict_va[0])/60),sum(sum(predict_va[1])/60))

C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\output\cat-mel_2bar_small_input1-extractions_2020-10-16_010019-000-of-005.wav


E:\anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:34: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


-0.3332752814749256 -0.18409335642354563
C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\output\cat-mel_2bar_small_input1-extractions_2020-10-16_010019-003-of-005.wav
-0.25157697290160286 -0.19144662251346745
C:\Users\Arnold\Desktop\4560\MusicVAE\DEAM\practice\output\cat-mel_2bar_small_input1-extractions_2020-10-16_010019-004-of-005.wav
-0.36904659878928214 -0.4114235248416662
